In [9]:
%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

fig = plt.figure(figsize=(20,30))

img=mpimg.imread('/home/matt/PycharmProjects/AirDrums/data/raw/04262022-130602/cam0_92.jpg')

def onclick(event):
    ix, iy = event.xdata, event.ydata
    print(ix, iy)

cid = fig.canvas.mpl_connect('button_press_event', onclick)

imgplot = plt.imshow(img)
plt.show()

pts = plt.ginput(4) #number of clicks
print(pts)

127.79078042700331 214.83191266937393
355.52589361103287 286.2403803626714
100.7713602187286 296.5334928229665
289.263982147883 357.00552852720034
[(127.79078042700331, 214.83191266937393), (355.52589361103287, 286.2403803626714), (100.7713602187286, 296.5334928229665), (289.263982147883, 357.00552852720034)]


In [6]:
import numpy as np
import csv

path = 'data/labelled/labelled_data.csv'
with open(path, 'r') as f:
    reader = csv.reader(f, delimiter=',')
    headers = next(reader)
    DATA = np.array(list(reader))

print(headers)

['filename', ' timestamp', ' right_tip_x', ' right_tip_y', ' left_tip_x', ' left_tip_y', ' right_hand_x', ' right_hand_y', ' left_hand_x', ' left_hand_y']


In [7]:
import torchvision.transforms as transforms
from dataset import XYDataset

TASK = 'AirDrums'

CATEGORIES = ['left_tip', 'right_tip', 'left_hand', 'right_hand']

DATASETS = ['A', 'B', 'C']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset('../data/regression/' + TASK + '_' + name, DATA, TRANSFORMS)

print("{} task with {} categories defined".format(TASK, CATEGORIES))

AirDrums task with ['left_tip', 'right_tip', 'left_hand', 'right_hand'] categories defined


In [8]:
import cv2
import ipywidgets
import traitlets
from IPython.display import display, Image
import os
# from jetcam.utils import bgr8_to_jpeg #, jpeg_to_bgr8
# from jupyter_clickable_image_widget import ClickableImageWidget
import glob

# pathname = 'data/raw/04262022-130602/*.jpg'
# image_list = glob.glob(pathname, recursive=False)
# image_list = sorted(image_list)
#
# image_list = np.genfromtxt('myfile.csv',delimiter=',')

directory = '/home/matt/PycharmProjects/AirDrums/data/raw/04262022-130602'

# initialize active dataset
dataset = datasets[DATASETS[0]]


path = '/home/matt/PycharmProjects/AirDrums/data/raw/04262022-130602/meta_cam0.csv'
with open(path, 'r') as f:
    reader = csv.reader(f, delimiter=',')
    image_head = next(reader)
    image_list = np.array(list(reader))

print(image_head)




['filename', 'timestamp']


In [10]:
%matplotlib tk
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

fig = plt.figure(figsize=(20,30))

data=None
rtipx=None
rtipy=None
ltipx=None
ltipy=None
rhandx=None
rhandy=None
lhandx=None
lhandy=None

for i in range(len(image_list)):

    img=mpimg.imread(os.path.join(directory,image_list[i][0]))

    def onclick(event):
        ix, iy = event.xdata, event.ydata
        print(ix, iy)

    cid = fig.canvas.mpl_connect('button_press_event', onclick)

    imgplot = plt.imshow(img)
    plt.show()

    pts = plt.ginput(4) #number of clicks
    print(pts)

    rtip = pts[0]
    rhand = pts[1]
    ltip = pts[2]
    lhand = pts[3]

    DATA = dataset.save_entry(filename=image_list[i][0], timestamp=image_list[i][1], rtip=rtip, ltip=ltip, rhand=rhand, lhand=lhand)

np.savetxt("data/labelled/labelled.csv", DATA, delimiter=",")


379.3287161754654 174.30278235696187
241.01501749024965 274.0173093160709
237.79841984640746 248.9278476941015
333.0097101041373 297.82013188050337
[(379.3287161754654, 174.30278235696187), (241.01501749024965, 274.0173093160709), (237.79841984640746, 248.9278476941015), (333.0097101041373, 297.82013188050337)]
386.4052309919183 193.60236822001525
386.4052309919183 193.60236822001525
232.00854408749137 291.3869365928189
232.00854408749137 291.3869365928189
235.22514173133368 248.9278476941015
235.22514173133368 248.9278476941015
[(333.0097101041373, 298.4634514092718), (386.4052309919183, 193.60236822001525), (232.00854408749137, 291.3869365928189), (235.22514173133368, 248.9278476941015)]
387.0485505206867 208.3987173816895
387.0485505206867 208.3987173816895
387.0485505206867 208.3987173816895
192.76605283261625 314.546439628483
192.76605283261625 314.546439628483
192.76605283261625 314.546439628483
250.66481042177634 239.92137429134328
250.66481042177634 239.92137429134328
250.66481

IndexError: list index out of range

In [18]:

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=dataset.categories, description='category')
count_widget = ipywidgets.IntText(description='count')

save_button = ipywidgets.Button(description='Save data')
nextimg_button = ipywidgets.Button(description='Next Image')

# def next_img(b):
#     global image_list, image_num
#     with open(image_list[image_num], 'rb') as file:
#         next_image = file.read()
#     snapshot_widget.value = next_image
#     image_num += 1
#     #camera = ipywidgets.Image(image_list[image_num])
#     #traitlets.dlink((camera, 'value'), (camera_widget, 'value'))
#
# nextimg_button.on_click(next_img)

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)


# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')



    global x, y, data
    # if content['event'] == 'click':
    #     data = content['eventData']
    #     x = data['offsetX']
    #     y = data['offsetY']
    #
    #     # save to disk
    #     # dataset.save_entry(category_widget.value, camera.value, x, y)
    #
    #     # display saved snapshot
    #     snapshot = snapshot_widget.copy().tobytes()
    #     snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
    #     snapshot_widget.value = bgr8_to_jpeg(snapshot)
    #     count_widget.value = dataset.get_count(category_widget.value)

def save_snapshot(b):
    global x, y, data
    if data:
        dataset.save_entry(category_widget.value, camera.value, x, y)


save_button.on_click(save_snapshot)
snapshot_widget.on_msg(display_snapshot)

data_collection_widget = ipywidgets.HBox([
    snapshot_widget,
    ipywidgets.VBox([ipywidgets.HBox([nextimg_button, save_button]),
    dataset_widget,
    category_widget,
    count_widget])])

display(data_collection_widget)
print("data_collection_widget created")

data_collection_widget created
